# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [6]:
# import libraries
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import re

from nltk import pos_tag 

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline 

from sklearn.model_selection import train_test_split

from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report 

from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# load data from database
engine = create_engine('sqlite:///FemaResponseData.db')

df = pd.read_sql_table('Messages_categories', engine) 

print(df.columns)

X = df.message

Y = df.drop(['id', 'message', 'original', 'genre', 'categories'], axis = 1) 


Index(['id', 'message', 'original', 'genre', 'categories', 'related',
       'request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')


### 2. Write a tokenization function to process your text data

In [8]:
stop = stopwords.words('english')

def tokenize(text):
    # remove punctutation, covert to lowercase, strip spaces, lemmatize, remove common words 
    # reduce case
    words = text.lower()
    # remove puntuatuion 
    words = re.sub('[^a-z0-9]', ' ', words)
    #split words into list 
    word_list = word_tokenize(words)
   
    #lemmatize
    Lemma = WordNetLemmatizer()
    
    token_list = []
    
    for x in word_list:
        
        if x not in stop:
          
            token = Lemma.lemmatize(x, 'v').strip()

            token_list.append(token)
  
    return token_list
  

In [9]:
car = "here we go. there IS a Car."

tokenize(car)

['go', 'car']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    
    ('tree', MultiOutputClassifier(DecisionTreeClassifier())), 
   
 ]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size =.25)

print(X_train.shape)

print(y_train.shape)

pipeline.fit(X_train, y_train)

#pipeline.score(X_test, y_test)

(19662,)
(19662, 36)


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
y_pred = pipeline.predict(X_test)

print(y_pred.shape)
print(y_test.shape)

for x in range(36):
    
    print(x+1,"Report:",classification_report(y_pred[:,x], np.array(y_test)[:,x]))
    print("Accuracy", (y_pred == y_test).mean())


(6554, 36)
(6554, 36)
1 Report:              precision    recall  f1-score   support

          0       0.50      0.53      0.51      1476
          1       0.86      0.84      0.85      5051
          2       0.29      0.52      0.37        27

avg / total       0.78      0.77      0.77      6554

Accuracy related                   0.770064
request                   0.855050
offer                     0.994355
aid_related               0.714678
medical_help              0.900671
medical_products          0.942936
search_and_rescue         0.962923
security                  0.970247
military                  0.959262
child_alone               1.000000
water                     0.954226
food                      0.939121
shelter                   0.935764
clothing                  0.986268
money                     0.968111
missing_people            0.984742
refugees                  0.959262
death                     0.959719
other_aid                 0.820110
infrastructure_related    

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.named_steps

{'tfidf': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words=None, strip_accents=None, sublinear_tf=False,
         token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=<function tokenize at 0x7f972c1dbd90>, use_idf=True,
         vocabulary=None),
 'tree': MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'),
            n_jobs=1)}

In [15]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'tfidf', 'tree', 'tfidf__analyzer', 'tfidf__binary', 'tfidf__decode_error', 'tfidf__dtype', 'tfidf__encoding', 'tfidf__input', 'tfidf__lowercase', 'tfidf__max_df', 'tfidf__max_features', 'tfidf__min_df', 'tfidf__ngram_range', 'tfidf__norm', 'tfidf__preprocessor', 'tfidf__smooth_idf', 'tfidf__stop_words', 'tfidf__strip_accents', 'tfidf__sublinear_tf', 'tfidf__token_pattern', 'tfidf__tokenizer', 'tfidf__use_idf', 'tfidf__vocabulary', 'tree__estimator__class_weight', 'tree__estimator__criterion', 'tree__estimator__max_depth', 'tree__estimator__max_features', 'tree__estimator__max_leaf_nodes', 'tree__estimator__min_impurity_decrease', 'tree__estimator__min_impurity_split', 'tree__estimator__min_samples_leaf', 'tree__estimator__min_samples_split', 'tree__estimator__min_weight_fraction_leaf', 'tree__estimator__presort', 'tree__estimator__random_state', 'tree__estimator__splitter', 'tree__estimator', 'tree__n_jobs'])

In [16]:
parameters = {
    #'tfidf__ngram_range': ((1,1),(1,2),(2,1),(2,2)),
    'tfidf__max_df': (1.0, 3.0, 10.0),
    'tfidf__max_features': (None,1000, 2500)      
             
    }
#n_jobs=4
cv = GridSearchCV(pipeline, param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__max_df': (1.0, 3.0, 10.0), 'tfidf__max_features': (None, 1000, 2500)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
cv.best_params_

{'tfidf__max_df': 3.0, 'tfidf__max_features': 1000}

In [19]:
y_preds_cv = cv.predict(X_test)

In [25]:
 print("Accuracy", (y_preds_cv == y_test).mean())

Accuracy related                   0.762893
request                   0.855966
offer                     0.992829
aid_related               0.704913
medical_help              0.902502
medical_products          0.942630
search_and_rescue         0.959872
security                  0.972078
military                  0.959872
child_alone               1.000000
water                     0.953311
food                      0.936527
shelter                   0.932103
clothing                  0.985505
money                     0.968264
missing_people            0.985810
refugees                  0.958804
death                     0.959109
other_aid                 0.815075
infrastructure_related    0.900977
transport                 0.942630
buildings                 0.949954
electricity               0.974519
tools                     0.992524
hospitals                 0.983674
shops                     0.993287
aid_centers               0.978181
other_infrastructure      0.936527
weather_rel

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# benchmark model

### 9. Export your model as a pickle file

In [51]:
import pickle 

pickle.dump(cv, open('/model1.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.